# Homework 1: Linear Regression

<font color=red>本次目标</font>：由前9个小时的18个features（包含PM2.5）预测的是个小时PM2.5

## Load 'train.csv'

train.csv 的资料为12个月中，每个月取20天，每天24小时的资料（每小时的资料有18个feature）<br/>
<font color=blue>赛题网址：https://www.kaggle.com/c/ml2020spring-hw1/data

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('./data/train.csv', encoding='big5')

## Preprocessing

取需要的数值部分，将"RAINFALL"栏位全部补0.另外，如果要在colab重复这段代码的执行，请从开头开始执行（把上面的都重新跑一次），以避免跑出的不是自己要的结果（若自己的代码不会遇到，但colab重复跑这一段会一直往下拉取资料，意即第一次取原本资料的第三次之后的资料，第二次取第一次取得资料掉三栏之后的资料）

In [3]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

In [4]:
data.head(18)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,16,9.2,8.2,6.9,6.8,3.8,6.9,7.8,15,21,...,11,11,22,28,19,12,8.1,7,6.9,6
6,17,9.8,8.7,8.6,8.5,5.3,8.8,9.9,22,29,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,16,30,27,23,24,28,24,22,21,29,...,65,64,51,34,33,34,37,38,38,36
8,56,50,48,35,25,12,4,2,11,38,...,52,51,66,85,85,63,46,36,42,42
9,26,39,36,35,31,28,25,20,19,30,...,36,45,42,49,45,44,41,30,24,13


In [5]:
raw_data.shape

(4320, 24)

## Extract Features(1)

将原始 4320\*18 的资料依照每个月份重组成12个18（features）*480（hours）的资料

解释4320<br/>
每个月取前20天进行训练，每天有18个特征，故4320/18=240天，240/20=12个月

In [6]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [7]:
month_data

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
        [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
        [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
        ...,
        [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
        [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
        [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]),
 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
        [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
        [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
        ...,
        [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
        [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
        [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]),
 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
        [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
        [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
        ...,
        [103.  , 128.  , 115.  , ...,  

## Extract Features(2)

每个月会有480hrs，每9个小时形成一个data，每个月会有471个data，故总资料数为471\*12笔，而每笔data有9\*18的features(一小时18个features * 9小时)。<br/>
对应的target则有 471 * 12 个（第10个小时的PM2.5）

In [8]:
x = np.empty([12 * 471, 18 * 9], dtype=float)
y = np.empty([12 * 471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:, day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) # vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] # value
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


## Normalize(1)

In [9]:
mean_x = np.mean(x, axis = 0) # 18 * 9
std_x = np.std(x, axis = 0) # 18 * 9
for i in range(len(x)): # 12 * 471
    for j in range(len(x[0])): # 18 * 9
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

## Split Training Data Into "train_set" and "validation_set"

这部分是针对作业中report的第二题、第三题做的简单示范，以生成比较中用来训练的train_set 和不会被放入训练、只是用来验证的validation_set

In [10]:
import math
x_train_set = x[: math.floor(len(x)*0.8), :]
y_train_set = y[: math.floor(len(y)*0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print('------------------------------------------------------------------')
print(x_validation)
print(y_validation)
print('------------------------------------------------------------------')

print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
------------------------------------------------------------------
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211


## Training

下面的code使用 <font color=red>Root Mean Square Error</font><br/>
因为常数项的存在，所以dimension(dim)需要多加一栏；eps项是避免adagrad的分母为0而加的极小数值
每一个dimension(dim)会对应到各自的gradient, weight(w)，透过一次次的iteration(item_time)学习

In [11]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 0.1
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
        
np.save('weight.npy', w)
w

0:27.071214829194115
100:21.471647263513006
200:20.551959842982743
300:19.890787864559883
400:19.35415962607147
500:18.89380815795257
600:18.486175780571084
700:18.117780717343162
800:17.780082268756605
900:17.467271225327377


array([[ 5.83999088e+00],
       [ 5.24343032e-03],
       [-5.13708801e-01],
       [-2.03113139e-01],
       [-4.92960388e-01],
       [ 6.42049067e-01],
       [-1.47887466e-01],
       [-1.82160988e-01],
       [-3.85173025e-01],
       [ 9.93226923e-01],
       [-1.58188342e-01],
       [ 5.56251334e-02],
       [ 7.08773938e-02],
       [-1.21467539e-02],
       [-9.12861026e-02],
       [-1.42664001e-02],
       [ 2.71361499e-02],
       [ 2.61459721e-01],
       [ 5.33995877e-01],
       [ 1.41828509e-01],
       [ 2.05236889e-02],
       [ 8.62999652e-02],
       [-1.67353590e-01],
       [ 3.28404572e-02],
       [-1.35110575e-01],
       [-5.48613118e-02],
       [ 2.96740297e-01],
       [ 6.63805095e-01],
       [-1.38581068e-01],
       [ 1.34841966e-01],
       [ 7.67817724e-02],
       [ 2.47749823e-01],
       [ 6.05739991e-02],
       [-2.15733289e-01],
       [-6.89653637e-02],
       [ 8.44078801e-03],
       [ 2.86133055e-01],
       [ 2.21895055e-01],
       [ 8.9

## Testing

载入test data，并且以相似与训练资料预先处理和特征选取的方式处理，使test data 形成240和维度为18*9+1的资料

In [12]:
testdata = pd.read_csv('./data/test.csv',header=None, encoding='big5')
test_data = testdata.iloc[:,2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18 * (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
            
test_x = np.concatenate((np.ones([240, 1]), test_x), axis=1).astype(float)
test_x

C:\Users\y308\Anaconda3\envs\tf_torch\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\y308\Anaconda3\envs\tf_torch\lib\site-packages\pandas\core\frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

## Prediction

有了weight和测试资料即可预测target

In [14]:
w = np.load('weight.npy')

ans_new = np.dot(test_x, w)
ans_new

array([[-6.41458374e+00],
       [ 3.83653240e+00],
       [ 1.44327616e+01],
       [-4.73562606e+00],
       [ 1.39312877e+01],
       [ 7.60527760e+00],
       [ 6.95657394e+00],
       [ 1.87147019e+01],
       [ 7.52775075e+00],
       [ 3.49120899e+01],
       [ 4.23989359e+00],
       [-5.06297161e+00],
       [ 3.82958627e+01],
       [ 3.34188258e+01],
       [ 2.81960018e+00],
       [-5.36764219e+00],
       [ 1.53161263e+01],
       [ 4.98918695e+01],
       [-1.33060318e+01],
       [-2.68982321e+00],
       [ 2.60896045e+01],
       [ 5.06059437e+01],
       [-7.03421099e+00],
       [ 5.16234533e+00],
       [-5.82424180e-01],
       [ 1.77159663e+01],
       [-4.88320335e+00],
       [ 5.35038938e+01],
       [-7.10380627e+00],
       [ 4.36598189e+01],
       [ 9.82183204e+00],
       [-8.10505743e+00],
       [-1.03207630e+01],
       [ 1.55007868e+01],
       [ 1.86461684e+01],
       [ 1.81318013e+01],
       [ 2.32745249e+01],
       [ 1.50306269e+01],
       [ 3.3

In [15]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[-6.41458374e+00],
       [ 3.83653240e+00],
       [ 1.44327616e+01],
       [-4.73562606e+00],
       [ 1.39312877e+01],
       [ 7.60527760e+00],
       [ 6.95657394e+00],
       [ 1.87147019e+01],
       [ 7.52775075e+00],
       [ 3.49120899e+01],
       [ 4.23989359e+00],
       [-5.06297161e+00],
       [ 3.82958627e+01],
       [ 3.34188258e+01],
       [ 2.81960018e+00],
       [-5.36764219e+00],
       [ 1.53161263e+01],
       [ 4.98918695e+01],
       [-1.33060318e+01],
       [-2.68982321e+00],
       [ 2.60896045e+01],
       [ 5.06059437e+01],
       [-7.03421099e+00],
       [ 5.16234533e+00],
       [-5.82424180e-01],
       [ 1.77159663e+01],
       [-4.88320335e+00],
       [ 5.35038938e+01],
       [-7.10380627e+00],
       [ 4.36598189e+01],
       [ 9.82183204e+00],
       [-8.10505743e+00],
       [-1.03207630e+01],
       [ 1.55007868e+01],
       [ 1.86461684e+01],
       [ 1.81318013e+01],
       [ 2.32745249e+01],
       [ 1.50306269e+01],
       [ 3.3

## Save Prediction to CSV File

In [16]:
import csv
with open('./data/sample_submission.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', -6.414583738037098]
['id_1', 3.8365324019217555]
['id_2', 14.432761564937705]
['id_3', -4.735626060577188]
['id_4', 13.931287721126296]
['id_5', 7.605277596636642]
['id_6', 6.95657394327743]
['id_7', 18.71470192412631]
['id_8', 7.527750751935662]
['id_9', 34.912089900106615]
['id_10', 4.239893589551889]
['id_11', -5.06297161066632]
['id_12', 38.29586274428263]
['id_13', 33.4188257513417]
['id_14', 2.819600181733542]
['id_15', -5.367642192409288]
['id_16', 15.316126326011897]
['id_17', 49.89186947157705]
['id_18', -13.306031849682515]
['id_19', -2.6898232083372546]
['id_20', 26.089604482775133]
['id_21', 50.60594374616168]
['id_22', -7.034210987258113]
['id_23', 5.162345334325305]
['id_24', -0.5824241799115082]
['id_25', 17.715966286861242]
['id_26', -4.883203346132694]
['id_27', 53.5038938450459]
['id_28', -7.103806268495858]
['id_29', 43.65981886528573]
['id_30', 9.821832035904647]
['id_31', -8.105057425799169]
['id_32', -10.320763012402578]
['id_33', 15.50078

In [18]:
dim = 17
learning_rate = np.array([[200]] * dim)
learning_rate

array([[200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200],
       [200]])